In [8]:
import os
import boto3
import re
import sagemaker
import pandas as pd 


role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# S3 bucket for training data.
# Feel free to specify a different bucket and prefix.
data_bucket = f"jumpstart-cache-prod-{region}"
data_prefix = "1p-notebooks-datasets/abalone/text-csv"


# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket and prefix
output_bucket = sagemaker.Session().default_bucket()
output_prefix = "workshop/linear-learner-abalone-regression"

In [9]:
print(f"Output bucket: {output_bucket}")

Output bucket: sagemaker-us-east-1-639556434474


In [11]:
FILE_TRAIN = "abalone_dataset1_train.csv"

s3 = boto3.client("s3")
s3.download_file(data_bucket, f"{data_prefix}/train/{FILE_TRAIN}", FILE_TRAIN)

df = pd.read_csv(FILE_TRAIN, sep=",", encoding="latin1", 
                 names=["age","sex","Length","Diameter","Height","Whole.weight","Shucked.weight","Viscera.weight","Shell.weight"]
                )

print(df.head(1))

   age  sex  Length  Diameter  Height  Whole.weight  Shucked.weight  \
0    8    2   0.615      0.48    0.16        1.2525           0.585   

   Viscera.weight  Shell.weight  
0          0.2595          0.33  


In [12]:
# creating the inputs for the fit() function with the training and validation location
s3_train_data = f"s3://{data_bucket}/{data_prefix}/train"
print(f"training files will be taken from: {s3_train_data}")

s3_validation_data = f"s3://{data_bucket}/{data_prefix}/validation"
print(f"validtion files will be taken from: {s3_validation_data}")

output_location = f"s3://{output_bucket}/{output_prefix}/output"
print(f"training artifacts output location: {output_location}")

training files will be taken from: s3://jumpstart-cache-prod-us-east-1/1p-notebooks-datasets/abalone/text-csv/train
validtion files will be taken from: s3://jumpstart-cache-prod-us-east-1/1p-notebooks-datasets/abalone/text-csv/validation
training artifacts output location: s3://sagemaker-us-east-1-639556434474/workshop/linear-learner-abalone-regression/output


In [13]:
# generating the session.s3_input() format for fit() accepted by the sdk
train_data = sagemaker.inputs.TrainingInput(
    s3_train_data,
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
    record_wrapping=None,
    compression=None,
)
validation_data = sagemaker.inputs.TrainingInput(
    s3_validation_data,
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
    record_wrapping=None,
    compression=None,
)

In [14]:
# getting the linear learner image according to the region
from sagemaker.image_uris import retrieve

container = retrieve("linear-learner", boto3.Session().region_name, version="1")
print(container)

382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:1


In [15]:
%%time
import boto3
import sagemaker
from time import gmtime, strftime

sess = sagemaker.Session()

job_name = "workshop-linear-regression-" + strftime("%H-%M-%S", gmtime())
print("Training job", job_name)

linear = sagemaker.estimator.Estimator(
    container,
    role,
    input_mode="File",
    instance_count=1,
    instance_type="ml.m5.xlarge",
    output_path=output_location,
    sagemaker_session=sess,
    tags=[
        {
            "Key":"Grupo",
            "Value":"Ecosistema Algoritmos"
        }
    ]
)

linear.set_hyperparameters(
    feature_dim=8,
    epochs=16,
    wd=0.01,
    loss="absolute_loss",
    predictor_type="regressor",
    normalize_data=True,
    optimizer="adam",
    mini_batch_size=100,
    lr_scheduler_step=100,
    lr_scheduler_factor=0.99,
    lr_scheduler_minimum_lr=0.0001,
    learning_rate=0.1,
)

Training job workshop-linear-regression-23-01-34
CPU times: user 38.8 ms, sys: 0 ns, total: 38.8 ms
Wall time: 47.9 ms


In [16]:
%%time
linear.fit(inputs={"train": train_data, "validation": validation_data}, job_name=job_name)

2021-03-14 23:01:52 Starting - Starting the training job...
2021-03-14 23:02:15 Starting - Launching requested ML instancesProfilerReport-1615762911: InProgress
.........
2021-03-14 23:03:36 Starting - Preparing the instances for training...
2021-03-14 23:04:17 Downloading - Downloading input data
2021-03-14 23:04:17 Training - Downloading the training image...
2021-03-14 23:04:46 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[03/14/2021 23:04:50 INFO 140529735182144] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_le

In [17]:
%%time
# creating the endpoint out of the trained model
linear_predictor = linear.deploy(initial_instance_count=1, instance_type="ml.c4.xlarge")
print(f"\ncreated endpoint: {linear_predictor.endpoint_name}")

---------------!
created endpoint: linear-learner-2021-03-14-23-25-27-768
CPU times: user 255 ms, sys: 22.6 ms, total: 277 ms
Wall time: 7min 32s


In [28]:
# configure the predictor to accept to serialize csv input and parse the reposne as json
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

linear_predictor.serializer = CSVSerializer()
linear_predictor.deserializer = JSONDeserializer()

In [48]:
%%time
import json
from itertools import islice
import math
import struct
import boto3
import random

# downloading the test file from data_bucket
FILE_TEST = "abalone_dataset1_test.csv"
s3 = boto3.client("s3")
s3.download_file(data_bucket, f"{data_prefix}/test/{FILE_TEST}", FILE_TEST)

# getting testing sample from our test file
test_data = [l for l in open(FILE_TEST, "r")]
sample = random.choice(test_data).split(",")
actual_age = sample[0]
payload = sample[1:]  # removing actual age from the sample
payload = ",".join(map(str, payload))

CPU times: user 30 ms, sys: 3.97 ms, total: 34 ms
Wall time: 167 ms


In [49]:
print(f"payload: {json.dumps(payload)}")

payload: "3,0.655,0.515,0.145,1.25,0.5265,0.283,0.315\n"


In [50]:
sample

['15', '3', '0.655', '0.515', '0.145', '1.25', '0.5265', '0.283', '0.315\n']

In [51]:
# Invoke the predicor and analyise the result
result = linear_predictor.predict(payload)

In [52]:
result = round(float(result["predictions"][0]["score"]), 2)

In [53]:
result

10.06

In [54]:
accuracy = str(round(100 - ((abs(float(result) - float(actual_age)) / float(actual_age)) * 100), 2))
print(f"Actual age: {actual_age}\nPrediction: {result}\nAccuracy: {accuracy}")

Actual age: 15
Prediction: 10.06
Accuracy: 67.07
